In [1]:
import json
import folium
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize


In [7]:
conn= http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')

In [8]:
covid1=json.loads(data)

In [9]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,43,38815,1,1426,25,32098,2020-09-16T18:29:30Z
1,Albania,AL,albania,152,11672,2,340,53,6668,2020-09-16T18:29:30Z
2,Algeria,DZ,algeria,238,48734,12,1632,181,34385,2020-09-16T18:29:30Z
3,Andorra,AD,andorra,0,1438,0,53,0,945,2020-09-16T18:29:30Z
4,Angola,AO,angola,130,3569,3,139,8,1332,2020-09-16T18:29:30Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,0,1063,0,35,5,931,2020-09-16T18:29:30Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-16T18:29:30Z
185,Yemen,YE,yemen,3,2016,0,583,4,1219,2020-09-16T18:29:30Z
186,Zambia,ZM,zambia,99,13819,4,324,210,12590,2020-09-16T18:29:30Z


In [10]:
df=pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,43,38815,1,1426,25,32098,2020-09-16T18:29:30Z,{}
1,Albania,AL,albania,152,11672,2,340,53,6668,2020-09-16T18:29:30Z,{}
2,Algeria,DZ,algeria,238,48734,12,1632,181,34385,2020-09-16T18:29:30Z,{}
3,Andorra,AD,andorra,0,1438,0,53,0,945,2020-09-16T18:29:30Z,{}
4,Angola,AO,angola,130,3569,3,139,8,1332,2020-09-16T18:29:30Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,0,1063,0,35,5,931,2020-09-16T18:29:30Z,{}
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-16T18:29:30Z,{}
185,Yemen,YE,yemen,3,2016,0,583,4,1219,2020-09-16T18:29:30Z,{}
186,Zambia,ZM,zambia,99,13819,4,324,210,12590,2020-09-16T18:29:30Z,{}


In [11]:
covid2= df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,43,38815,1,1426,25,32098
1,Albania,152,11672,2,340,53,6668
2,Algeria,238,48734,12,1632,181,34385
3,Andorra,0,1438,0,53,0,945
4,Angola,130,3569,3,139,8,1332
...,...,...,...,...,...,...,...
183,Viet Nam,0,1063,0,35,5,931
184,Western Sahara,0,10,0,1,0,8
185,Yemen,3,2016,0,583,4,1219
186,Zambia,99,13819,4,324,210,12590


In [12]:
m=folium.Map(tiles='Stamen Terrain',min_zoom=1.5)
m

In [13]:
url='http://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes= f'{url}/world-countries.json'


In [21]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid2,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

In [22]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [23]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [38]:
covid_final=pd.merge(covid2,coordinates,on='Country')

AttributeError: module 'folium' has no attribute 'CircleMaker'

In [40]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                      radius=5,
                      weight=2,
                      popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                      fill_color='#000000').add_to(m)

In [41]:
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m

In [44]:
ml= folium.Map(tiles='StamenToner',min_zoom=2)
ml

In [45]:
deaths=covid_final['TotalDeaths'].astype(float)

In [46]:
lat=covid_final['latitude'].astype(float)

In [47]:
lon=covid_final['longitude'].astype(float)

In [48]:
ml.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

In [54]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalDeaths],
                       fill_color='#000000').add_to(ml)
covid_final.apply(plotDot,axis=1)
ml.fit_bounds(ml.get_bounds())
ml